In [40]:
import random
import numpy as np
import tensorflow as tf

seed_value= 1111
random.seed(seed_value)
np.random.seed(seed_value)
tf.set_random_seed(seed_value)

import warnings
import sys
import os
import pandas as pd
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
# from sklearn.metrics import mean_absolute_percentage_error
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential,Model
from keras.layers import Input,LSTM, Dense, Flatten, Conv1D, Lambda, Reshape, RepeatVector
from keras.layers.merge import concatenate, multiply,add
from keras import regularizers
# from keras.initializers import gloroadasdat_uniform
from tqdm import tqdm
from keras import regularizers
from keras.models import load_model
import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import random as python_random
from statsmodels.tsa.stattools import pacf,acf
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf

In [41]:
# def MSE(pred,gt):
#     l = pred.shape[1]
#     err = np.zeros((l))
#     err2 = np.zeros((l))
# #     gt = gt[-1440:,:]
#     for i in range(l):
#         err[i] = mse(pred[:,i],gt[:,i])
#         err2[i] = mae(pred[:,i],gt[:,i])
#     return err,err2

def MSE(pred,gt):
    l = pred.shape[0]
    err = np.zeros((l))
    err2 = np.zeros((l))
#     gt = gt[-1440:,:]
#     for i in range(l):
    err[i] = mse(pred,gt)
    err2[i] = mae(pred,gt)
    return err,err2

def metrics(pred,gt):
    l = pred.shape[1]
#     print(l)
    err_mse = np.zeros((l))
    err_mae = np.zeros((l))

    for i in range(l):
        err_mse[i] = mse(pred[:,i],gt[:,i])
        err_mae[i] = mae(pred[:,i],gt[:,i])
        
    return err_mse,err_mae        

In [42]:
#------------------data loading--------------

d = "./dataset/" #---should be updated to match the path

horizon = 1
window_size=50

data = np.asarray(pd.read_csv(d+"traffic_30_with_std.csv",usecols=['avg_flow']))


In [43]:
#----train/test split---------------
train_d=data[:(int(.8*data.size)+1)]
n_test = int(.1*data.size)


In [44]:


def make_input(data,window_size,horizon=1):
    length=data.shape[0]
    data=np.append(data,0)
    y = np.zeros([length-window_size+1,horizon])
    output=np.zeros([length-window_size+1,window_size])
    for i in range(length-window_size+1):
        output[i:i+1,:]=data[i:i+window_size]
        y[i,:]= data[i+window_size:i+window_size+horizon]
#         print(i)
    return output.reshape(output.shape[0],window_size), y

def make_k_input(data,horizon):
    length = data.shape[0]
    output= np.zeros([length+1-horizon,horizon])
    for i in range(length-window_size-horizon+1):
        output[i:i+1,:]=data[i:i+horizon]
    return output.reshape(output.shape[0],horizon)

def nonov_make_input(data,window_size,horizon=1):
    length=data.shape[0]-window_size
    loop=length//horizon
    extra = length%horizon

    data = np.append(data,np.zeros([horizon-extra]))

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
        
    output=np.zeros([i_val,window_size])
    y=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[i*horizon:(i*horizon)+window_size]
        y[i,:]= data[(i*horizon)+window_size:(i*horizon)+window_size+horizon]
        
    return output.reshape(output.shape[0],window_size), y

def nonov_make_k_input(data,horizon):
    length = data.shape[0]
    loop=length//horizon
    extra = length%horizon
    data_app = np.repeat(data[-1],extra)
    data = np.append(data,data_app)    

    if extra ==0:
        i_val = loop
    else:
        i_val=loop+1
    output=np.zeros([i_val,horizon])
    for i in range(i_val):
        output[i:i+1,:]=data[(i*horizon):(i*horizon)+horizon]
    return output.reshape(output.shape[0],horizon)

def logic_rules(data,preds):
    day_sample=48
    window=8
    data=data[2:]
    loop=preds.size//window
    rem=preds.size%window
    out_data=np.zeros((loop,window))
    out_pred=np.zeros((loop,window))
    for i in range(loop):
        out_data[i,:]=data[i*window:i*window+window]
        out_pred[i,:]=preds[i*window:i*window+window]
    rem_data=data[-(48+rem):-48]
    rem_pred=preds[-rem:]
    return out_data,out_pred,rem_data,rem_pred
    
        

In [37]:
horizon=1
window_size=50
final_output=np.zeros((train_d.size-window_size))


p_acf = pacf(train_d,100)[1:]
indexes = np.squeeze(np.argwhere(np.abs(p_acf)>0.2))

weightage=np.ndarray.flatten(np.abs(p_acf[indexes]))
# weightage = weightage/np.sum(np.abs(weightage))
size_of_weightage= weightage.size
input_, label = make_input(data,window_size,horizon)
output =np.zeros([input_.shape[0],horizon])
weight_matrix=np.zeros([1,window_size])
weight_matrix[0,indexes]=weightage
# output = np.append(input_[:,-size_of_weightage:],output,axis=1)   
for z in range(input_.shape[0]):
#         temp=np.zeros{horizon}
#     for j in range(horizon):
#             print(output[z,j:j+size_of_weightage]*weightage)
#         output[z,j+size_of_weightage]= np.sum((output[z,j:j+size_of_weightage]*np.flip(weightage))/np.sum(weightage))
    output[z,0]= (np.sum(input_[z,:]*np.flip(weight_matrix)))/np.sum(weightage)

# output=output[:,size_of_weightage:]
temp=np.squeeze(np.ndarray.flatten(output))
final_output=temp


#-------------logic_rules---------------------


            

        
            
        
        
    

In [34]:
pred_mean= np.mean(out_pred,axis=1).reshape(out_pred.shape[0],1)
data_mean=np.mean(out_data,axis=1).reshape(out_data.shape[0],1)
pred_std=np.std(out_pred,axis=1)
data_std=np.std(out_data,axis=1)
diff=np.abs(pred_mean-data_mean)
a=np.zeros(0)
f_pred=out_pred
for i in range(out_pred.shape[0]):
    if diff[i]<0.1*data_std[i]:
        a=np.append(a,i)
        f_pred[i] = out_pred[i,:]-pred_mean[i,:]+data_mean[i,:]
# f_rem=pred_rem-np.mean(pred_rem)+np.mean(data_rem)
f_rem=pred_rem
temp1=np.ndarray.flatten(f_pred)
temp1=np.append(temp1,f_rem)

a=a[1:]
    


In [190]:
np.savetxt('/home/chatta/logic_rules/results/pacf_preds/traffic_30_std/h1/preds_final.csv',temp[1:], fmt='%1.5f',delimiter=',')